In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import cv2

import time

import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, '..')

from web_interaction.main import open_game, start_game, wait_loading, insert_user_stories_from_image
from game.game import ProductOwnerGame
from game.userstory_card.userstory_card_info import UserStoryCardInfo
from environment.backlog_env import BacklogEnv
from environment.environment import ProductOwnerEnv
from environment.reward_sytem.base_reward_system import BaseRewardSystem
from environment.userstory_env import UserstoryEnv
import image_parser

from pipeline.study_agent import load_dqn_agent
from single_color_storage import SingleColorStorage

In [3]:
driver = open_game()

In [4]:
iframe = driver.find_element(by=By.ID, value='game_drop')
height = iframe.rect["height"]  # 540
width = iframe.rect["width"]  # 960

In [5]:
wait_loading(iframe)

In [6]:
start_game(driver, iframe)

In [7]:
iframe.screenshot('iframe.png')

True

In [8]:
image = cv2.imread('iframe.png')
meta_info = image[7:83, 57:932]

In [9]:
game = ProductOwnerGame()
context = game.context

context.is_new_game = False

In [11]:
available_developers_count = 2
current_rooms_counter = 1
blank_sprint_counter = 0

In [12]:
current_sprint_hours = image_parser.get_current_sprint_hours(image)
current_sprint_hours

0.0

In [13]:
insert_user_stories_from_image(game, image)

In [14]:
game.userstories.stories_list

In [39]:
current_sprint = image_parser.get_sprint_number(meta_info)
context.current_sprint = current_sprint

money = image_parser.get_game_money(meta_info)
context.set_money(money)

loyalty = image_parser.get_loyalty(meta_info)
context.set_loyalty(loyalty)

context.customers = image_parser.get_customers(meta_info)

In [40]:
credit = max(300_000 - (current_sprint - 1) * 9_000, 0)
context.credit = credit
credit

273000.0

In [11]:
backlog_cards = image_parser.get_backlog(image)
backlog_cards

[(array([120,  79, 240], dtype=uint8), 17.0),
 (array([120,  79, 240], dtype=uint8), 8.0),
 (array([120,  79, 240], dtype=uint8), 8.0),
 (array([120,  79, 240], dtype=uint8), 5.0)]

In [ ]:
# gruop cards by color
# find user story card
# clear user story related cards
# fill user story with new related cards

In [17]:
from operator import itemgetter
from itertools import groupby
from typing import Tuple, Sequence, FrozenSet

from game.backlog_card.card_info import CardInfo
from game.userstory_card.userstory_card import UserStoryCard


def key_func(card: Tuple[list, float]):
    return frozenset(enumerate(card[0]))


def find_user_story_by_color(
    user_stories: Sequence[UserStoryCard], color: FrozenSet[Tuple[int, int]]
):
    for user_story in user_stories:
        if frozenset(enumerate(user_story.info.color)) == color:
            return user_story

backlog_cards_by_color = {}
for key, group in groupby(backlog_cards, key_func):
    backlog_cards[key] = list(group)

for user_story in game.userstories.release:
    info = user_story.info
    us_id_val = id(info)
    info.related_cards.clear()
    color = frozenset(enumerate(info.color))
    backlog_cards = backlog_cards_by_color[color]

    print(color)
    for _, hours in backlog_cards:
        print(hours)
        card_info = CardInfo(
            hours_val=hours,
            color_val=key,
            us_id_val=us_id_val,
            label_val=user_story.info.label,
            card_type_val=user_story.info.card_type,
        )
        info.related_cards.append(card_info)

frozenset({(1, 79), (0, 120), (2, 240)}) <itertools._grouper object at 0x000001ED7646B310>
(array([120,  79, 240], dtype=uint8), 17.0)
(array([120,  79, 240], dtype=uint8), 8.0)
(array([120,  79, 240], dtype=uint8), 8.0)
(array([120,  79, 240], dtype=uint8), 5.0)


In [17]:
userstory_env = UserstoryEnv(2, 0, 0)
backlog_env = BacklogEnv(6, 0, 0, 6, 0, 0)
reward_system = BaseRewardSystem(config={})
env = ProductOwnerEnv(userstory_env, backlog_env, with_info=True, reward_system=reward_system)
env.game = game

In [27]:
state = env._get_state()
state

array([ 4.  ,  0.33, 25.  ,  4.  ,  2.73,  2.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  1.  ,  0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ], dtype=float32)

In [28]:
info = env.get_info()
info

{'actions': [0, 1]}

In [20]:
agent = load_dqn_agent('../models/credit_start_model.pt')

In [29]:
action = agent.get_action(state, info)
action

1

In [23]:
card_info = userstory_env.userstories_common[action - 7].info
card_info.color

array([120,  79, 240], dtype=uint8)

In [24]:
card_info.position

(725, 183)

In [25]:
time.sleep(5)
# move user story
x, y = card_info.position
ActionChains(driver).move_to_element_with_offset(
    iframe, x - width // 2 + 10, y - height // 2 + 5
).click().perform()

In [26]:
env._perform_action(action)
env._get_state()

array([ 4.  ,  0.33, 25.  ,  4.  ,  2.73,  2.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  1.  ,  0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ], dtype=float32)

In [30]:
time.sleep(5)
# decompose
ActionChains(driver).move_to_element_with_offset(
    iframe, 817 - width // 2, 480 - height // 2
).click().perform()

In [13]:
iframe.rect

{'height': 540, 'width': 960, 'x': 29.600000381469727, 'y': 20}

In [15]:
driver.quit()